In [ ]:
import torch
print('torch.cuda.is_available(): {}'.format(torch.cuda.is_available()))
import numpy as np
from utils import make_imb_data, get_cifar10, plot_distribution
from models.wrn import WRN
from collections import Counter
import matplotlib.pyplot as plt
import time

In [ ]:
N_SAMPLES_PER_CLASS = make_imb_data(1250, 10, 100 ,False, False)
U_SAMPLES_PER_CLASS = make_imb_data(3 * 1250, 10, 100, False, False)
N_SAMPLES_PER_CLASS_T = torch.Tensor(N_SAMPLES_PER_CLASS)

train_labeled_set, train_unlabeled_set, test_set = get_cifar10('/home/apoorva/Datasets', N_SAMPLES_PER_CLASS
, U_SAMPLES_PER_CLASS)

In [ ]:
def create_model(ema=False):
    model = WRN(2, len(N_SAMPLES_PER_CLASS))
    model = model.cuda()

    if ema:
        for param in model.parameters():
            param.detach_()

    return model

model = create_model()
ema_model = create_model(ema=True)


In [ ]:
from sampler import UnlabeledDataLoader
target_dist = torch.ones([10, 10])
labeled_loader = torch.utils.data.DataLoader(train_labeled_set, batch_size=128, num_workers=8, shuffle=True)
for imgs, lbls, _ in labeled_loader:
    ulb_loader = UnlabeledDataLoader(train_unlabeled_set, model, target_dist, N_SAMPLES_PER_CLASS, 128)
    img_u, ulb_u = ulb_loader.get_batch(lbls)